In [6]:
import os
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from path import Path
import numpy as np
import hvplot.pandas
from newsapi import NewsApiClient
#from ibm_watson import ToneAnalyzerV3
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import plotly.express as px
import matplotlib.pyplot as plt
import matplotlib as mpl


%matplotlib inline

## Fetching latest News

In [7]:
#from newsapi import NewsApiClient
from newsapi.newsapi_client import NewsApiClient

In [8]:
# Read your api key environment variable
api_key = os.getenv('NEWS_API_KEY')

In [9]:
# Create a newsapi client
newsapi = NewsApiClient(api_key=api_key)

### Change date and thicker if needed

In [10]:
# Fetch the news articles
thicker = "Valero"
start_date = "2019-10-12"
end_date = "2019-11-11"
news = newsapi.get_everything(q=thicker, language="en", from_param= start_date, to= end_date ,sort_by="relevancy")
news["totalResults"]

TypeError: expected string or bytes-like object

In [36]:
analyzer = SentimentIntensityAnalyzer()

### fuction to get sentiment

In [37]:
def get_sentiment_scores(text, date, source, url):
    sentiment_scores = {}

    # Sentiment scoring with VADER
    text_sentiment = analyzer.polarity_scores(text)
    sentiment_scores["date"] = date
    sentiment_scores["text"] = text
    sentiment_scores["source"] = source
    sentiment_scores["url"] = url
    sentiment_scores["compound"] = text_sentiment["compound"]
    sentiment_scores["pos"] = text_sentiment["pos"]
    sentiment_scores["neu"] = text_sentiment["neu"]
    sentiment_scores["neg"] = text_sentiment["neg"]
    if text_sentiment["compound"] >= 0.05:  # Positive
        sentiment_scores["normalized"] = 1
    elif text_sentiment["compound"] <= -0.05:  # Negative
        sentiment_scores["normalized"] = -1
    else:
        sentiment_scores["normalized"] = 0  # Neutral

    return sentiment_scores


### Create for loop for DF

In [38]:
sentiments_data = []

# Loop through all the news articles
for article in news["articles"]:
    try:
        # Get sentiment scoring using the get_sentiment_score() function
        sentiments_data.append(
            get_sentiment_scores(
                article["content"],
                article["publishedAt"][:10],
                article["source"]["name"],
                article["url"],
            )
        )

    except AttributeError:
        pass

news_df = pd.DataFrame(sentiments_data)
news_df = news_df.sort_values(by="date")
news_df.set_index("date", inplace=True)
news_df.head()

,compound,neg,neu,normalized,pos,source,text,url
date,,,,,,,,
2019-10-13,0.8402,0.000,0.795,1,0.205,The Next Web,IBM today announced the 2019 Call for Code gra...,https://thenextweb.com/artificial-intelligence...
2019-10-13,-0.7430,0.161,0.839,-1,0.000,Wtap.com,HOUSTON (KTRK/CNN) - Police in Houston are sea...,https://www.wtap.com/content/news/Texas-woman-...
2019-10-13,-0.0772,0.151,0.741,-1,0.108,Yahoo.com,"Some say volatility, rather than debt, is the ...",https://finance.yahoo.com/news/valero-energy-n...
2019-10-15,-0.2960,0.045,0.955,-1,0.000,Si.com,Este fin de semana hubo actividad de la Fecha ...,https://www.si.com/soccer/2019/10/15/marchesin...
2019-10-23,0.4939,0.048,0.837,1,0.115,Si.com,Inter got their Champions League campaign back...,https://www.si.com/soccer/2019/10/23/inter-2-0...


In [39]:
news_df.describe()

,compound,neg,neu,normalized,pos
count,20.000000,20.000000,20.000000,20.00000,20.000000
mean,0.336280,0.040750,0.843200,0.50000,0.116050
std,0.453504,0.055072,0.085796,0.82717,0.078074
min,-0.743000,0.000000,0.699000,-1.00000,0.000000
25%,0.000000,0.000000,0.795750,0.00000,0.057250
50%,0.410550,0.000000,0.837500,1.00000,0.138000
75%,0.740650,0.062000,0.869500,1.00000,0.167500
max,0.883400,0.161000,1.000000,1.00000,0.260000


In [40]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re
import nltk
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rolop\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [41]:
def tokenizer(text):
    
    #create list
    text = word_tokenize(text)
    
    #covert to lowercase
    text = [word.lower() for word in text]
    
    #remove punctuation
    regex = re.compile("[^a-zA-Z ]")
    text = [regex.sub('', word) for word in text]
    
    # Remove the stop words
    sw = set(stopwords.words('english'))
    
    # Lemmatize Words into root words
    lem = WordNetLemmatizer() #root words
    text = [lem.lemmatize(word) for word in text]
    text = [word for word in text if word not in sw]
    
    return text

In [42]:
news_df.raw_article

AttributeError: 'DataFrame' object has no attribute 'raw_article'

In [21]:
news_token = []
[news_token.append(tokenizer(text)) for text in news_df.raw_article]   
news_df['Token'] = news_token
news_df.head()

,raw_article,compound,positive,negative,neutral,Token
0,IBM today announced the 2019 Call for Code gra...,0.8402,0.205,0.000,0.795,"[ibm, today, announced, , call, code, grand, p..."
1,Image source: The Motley Fool.\r\nValero Energ...,-0.2023,0.064,0.088,0.848,"[image, source, , motley, fool, , valero, ener..."
2,Valero Energy(NYSE:VLO) is one of the largest ...,0.4599,0.136,0.061,0.802,"[valero, energy, , nyse, , vlo, , one, largest..."
3,Courtesy: Houston Chronicle - Image: Corpus Ch...,0.6908,0.153,0.000,0.847,"[courtesy, , houston, chronicle, , image, , co..."
4,Here's how our experts see the bowl matchups a...,0.0000,0.000,0.000,1.000,"[expert, see, bowl, matchup, week, , , check, ..."
